In [2]:
from neo4j import GraphDatabase

In [15]:
driver = GraphDatabase.driver("bolt://localhost", auth=("neo4j", "root"),encrypted=False)

In [16]:
session=driver.session()

In [19]:
query="""LOAD CSV WITH HEADERS FROM 'file:///transactions-2020-12-03.csv' AS line
        MERGE (c:Customer {name:line.Customer_name, age:line.Customer_Age,id:line.loginID,occupation:line.Customer_Occupation,creditcard:line.Credit_Card_Number})
        MERGE (p:Product {name:line.Product_Name, productid:line.Product_ID,productCategory:line.Category})
        MERGE (m:Manufacturer {name:line.Manufacturer})
        MERGE (o:Order {orderID:line.Order_ID,orderDate:line.Order_Date})
        MERGE (d:Delivery {deliveryID:line.Delivery_Tracking_ID,deliveryDate:line.Expected_Delivery_Date,deliveryType:line.Delivery_Type,deliveryZip:line.Delivery_Zip_Code}) 
        
        MERGE (c)-[:BOUGHT {bought_on:line.Order_Date}]->(p)
        MERGE (m)-[:FIELD]->(p)
        MERGE (c)-[:PLACED_ORDER {order_rating:line.Review_Rating,transaction_status:line.Transaction_Status,order_return:line.Order_Returned,order_on_time:line.Order_Delivered_on_Time}]->(o)
        MERGE (o)-[:HAS_DELIVERY]->(d)
        """

In [20]:
session.run(query)

In [ ]:
#1.Get the number of transactions that were NOT delivered on time for every Delivery Zip Code 
MATCH (customer:Customer)-[transaction:PLACED_ORDER]->(order) 
WHERE transaction.order_rating = "5"
MATCH (customer)-[:HAS_DELIVERY]->(delivery)
WITH customer, order, transaction,delivery
RETURN DISTINCT delivery.deliveryZip AS `DELIVERY ZIP CODES`, count(DISTINCT transaction)
AS `Not Delivered Transactions`

In [ ]:
#1.Get the number of transactions that were NOT delivered on time for every Delivery Zip Code 
MATCH (customer:Customer)-[transaction:PLACED_ORDER]->(order) 
WHERE transaction.order_on_time = "1"
MATCH (order)-[:HAS_DELIVERY]->(delivery)
WITH customer, order, transaction,delivery
RETURN DISTINCT delivery.deliveryZip AS `DELIVERY ZIP CODES`, count(DISTINCT transaction)
AS `Not Delivered Transactions`

In [ ]:
#2.Get the number of disputed transactions and the list of customer names for these disputed transactions for Delivery Zip Code
MATCH (customer:Customer)-[transaction:PLACED_ORDER]->(order) 
WHERE transaction.transaction_status ="1"
MATCH (order)-[:HAS_DELIVERY]->(delivery)
WITH customer, order, transaction,delivery
RETURN DISTINCT delivery.deliveryZip AS `DELIVERY ZIP CODES`, count(DISTINCT transaction)
AS Count, collect(DISTINCT customer.name) AS Customer

In [ ]:
#3.Get the number of disputed transactions for EVERY CUSTOMER that has more that one disputed transaction 
MATCH (customer:Customer)-[transaction:PLACED_ORDER]->(order)
WHERE transaction.transaction_status = "1"
WITH customer, count(*) AS number_of_disputed_transactions
WHERE number_of_disputed_transactions > 1
RETURN customer.name AS Customer, number_of_disputed_transactions 

In [ ]:
#4.Get the top 3 customers that reported the maximum number of returned orders 
MATCH (customer:Customer)-[transaction:PLACED_ORDER]->(order)
WHERE transaction.order_return = "1"
WITH customer, count(*) AS number_of_returned_transactions
WHERE number_of_returned_transactions > 0
WITH customer, number_of_returned_transactions
ORDER BY number_of_returned_transactions DESC LIMIT 3
RETURN customer.name AS Customer, number_of_returned_transactions

In [ ]:
#5.Get the number of returned orders and got review rating 1 for every product category in every Delivery Zip Code
MATCH (customer:Customer)-[transaction:PLACED_ORDER]->(order)
WHERE transaction.order_return = "1" AND transaction.order_rating = '1'
MATCH (order)-[:HAS_DELIVERY]->(delivery)
WITH customer, order, transaction,delivery
RETURN DISTINCT delivery.deliveryZip AS `DELIVERY ZIP CODES`, count(DISTINCT transaction)
AS `Not Delivered Transactions`, transaction.order_rating AS `Review Rating`